In [28]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [9]:
os.getcwd()

'c:\\Users\\nktun\\OneDrive\\Documents\\GitHub\\nkt1850\\Spotify Data Analysis'

Read 4 json files and transform them into DataFrames, then concat into 1 final DataFrame (df)

In [13]:
df_part_1 = pd.read_json("Spotify Extended Streaming History\Streaming_History_Audio_2020-2023_0.json")
df_part_2 = pd.read_json("Spotify Extended Streaming History\Streaming_History_Audio_2023_1.json")
df_part_3 = pd.read_json("Spotify Extended Streaming History\Streaming_History_Audio_2023-2024_2.json")
# df_part_4 = pd.read_json("Spotify Extended Streaming History\Streaming_History_Video_2020-2024.json")

In [30]:
# Concat all given DataFrames into 1 DF for further analyses
spotify_df = pd.concat([df_part_1, df_part_2, df_part_3])
# Removing username column because I am the only one using this account
spotify_df.drop(columns=['username'], inplace=True)

In [31]:
spotify_df.head()

,ts,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2020-08-22T16:34:56Z,Windows 10 (10.0.18363; x64; AppX),238360,VN,171.241.150.149,unknown,unravel,TK from Ling tosite sigure,Fantastic Magic,spotify:track:0GLyqTysS0DFoeKjURtB8s,None,None,None,clickrow,trackdone,False,NaN,False,1598113859206,False
1,2020-08-22T16:40:32Z,Windows 10 (10.0.18363; x64; AppX),9490,VN,171.241.150.149,unknown,katharsis,TK from Ling tosite sigure,Katharsis,spotify:track:0PnrdoLqdpScsUTYd06Zb8,None,None,None,trackdone,logout,False,NaN,False,1598114096805,False
2,2020-08-22T16:40:42Z,Windows 10 (10.0.18363; x64; AppX),2500,VN,171.241.150.149,unknown,Dancing With A Stranger (with Normani),Sam Smith,Dancing With A Stranger (with Normani),spotify:track:6Qs4SXO9dwPj5GKvVOv8Ki,None,None,None,clickrow,endplay,False,NaN,False,1598114439494,False
3,2020-08-22T16:40:45Z,Windows 10 (10.0.18363; x64; AppX),3100,VN,171.241.150.149,unknown,How Do You Sleep?,Sam Smith,How Do You Sleep?,spotify:track:6b2RcmUt1g9N9mQ3CbjX2Y,None,None,None,clickrow,endplay,False,NaN,False,1598114442297,False
4,2020-08-22T16:40:52Z,Windows 10 (10.0.18363; x64; AppX),6580,VN,171.241.150.149,unknown,Too Good At Goodbyes,Sam Smith,The Thrill Of It All,spotify:track:1mXVgsBdtIVeCLJnSnmtdV,None,None,None,clickrow,endplay,False,NaN,False,1598114445537,False


In [32]:
music_df = spotify_df.drop(columns=['episode_name', 'episode_show_name', 'spotify_episode_uri'],inplace=False)
# Using the Music DF to only concern about music listened during my usage of Spotify from 2020 upto now

### Descriptive statistics of listening habit

In [39]:
print("Total tracks listened:", len(music_df))
print("Number of artists listened to:", music_df['master_metadata_album_artist_name'].nunique())
print("Number of unique songs listened to:", music_df['master_metadata_track_name'].nunique())

Total tracks listened: 41162
Number of artists listened to: 2876
Number of unique songs listened to: 7531


In [40]:
print("Top 10 artists of choice:", music_df['master_metadata_album_artist_name'].value_counts(sort=Descending)).head(10)

NameError: name 'Descending' is not defined